<a href="https://colab.research.google.com/github/Tclack88/misc-analysis/blob/main/vietnamese_frequency_sentence_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget "https://github.com/qhungngo/EVBCorpus/raw/master/EVBCorpus_EVBNews_v2.0.rar"
!ls

--2021-12-21 19:33:39--  https://github.com/qhungngo/EVBCorpus/raw/master/EVBCorpus_EVBNews_v2.0.rar
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/qhungngo/EVBCorpus/master/EVBCorpus_EVBNews_v2.0.rar [following]
--2021-12-21 19:33:39--  https://raw.githubusercontent.com/qhungngo/EVBCorpus/master/EVBCorpus_EVBNews_v2.0.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7310993 (7.0M) [application/octet-stream]
Saving to: ‘EVBCorpus_EVBNews_v2.0.rar’

EVBCorpus_EVBNews_v 100%[===================>]   6.97M  --.-KB/s    in 0.08s   

2021-12-21 19:33:40 (86.2 MB/s) - ‘EVBCorpus_EVBNews_v2.0.rar’ s

In [5]:
!unrar e "EVBCorpus_EVBNews_v2.0.rar"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from EVBCorpus_EVBNews_v2.0.rar

Extracting  N0001.sgml                                                     0%  OK 
Extracting  N0002.sgml                                                     0%  OK 
Extracting  N0003.sgml                                                     0%  OK 
Extracting  N0004.sgml                                                     0%  OK 
Extracting  N0005.sgml                                                     0%  OK 
Extracting  N0006.sgml                                                     0%  OK 
Extracting  N0007.sgml                                                     0%  OK 
Extracting  N0008.sgml                                                     1%  OK 
Extracting  N0009.sgml                                                     1%  OK 
Extracting  N0010.sgml                                     

In [1]:
!ls

EVBCorpus_EVBNews_v2.0.rar  N0201.sgml	N0402.sgml  N0603.sgml	N0804.sgml
N0001.sgml		    N0202.sgml	N0403.sgml  N0604.sgml	N0805.sgml
N0002.sgml		    N0203.sgml	N0404.sgml  N0605.sgml	N0806.sgml
N0003.sgml		    N0204.sgml	N0405.sgml  N0606.sgml	N0807.sgml
N0004.sgml		    N0205.sgml	N0406.sgml  N0607.sgml	N0808.sgml
N0005.sgml		    N0206.sgml	N0407.sgml  N0608.sgml	N0809.sgml
N0006.sgml		    N0207.sgml	N0408.sgml  N0609.sgml	N0810.sgml
N0007.sgml		    N0208.sgml	N0409.sgml  N0610.sgml	N0811.sgml
N0008.sgml		    N0209.sgml	N0410.sgml  N0611.sgml	N0812.sgml
N0009.sgml		    N0210.sgml	N0411.sgml  N0612.sgml	N0813.sgml
N0010.sgml		    N0211.sgml	N0412.sgml  N0613.sgml	N0814.sgml
N0011.sgml		    N0212.sgml	N0413.sgml  N0614.sgml	N0815.sgml
N0012.sgml		    N0213.sgml	N0414.sgml  N0615.sgml	N0816.sgml
N0013.sgml		    N0214.sgml	N0415.sgml  N0616.sgml	N0817.sgml
N0014.sgml		    N0215.sgml	N0416.sgml  N0617.sgml	N0818.sgml
N0015.sgml		    N0216.sgml	N0417.sgml  N0618.sgml	N0819.sgml
N0016.sgml		

In [2]:
!pip install pandas==1.3.0
from pathlib import Path
import pandas as pd
# pwd = Path('.')
file1 = Path('N0001.sgml')

In [3]:
pip install sgmllib3k

In [4]:
import sgmllib

In [5]:
class FoundTitle(Exception):
    pass

class ExtractTitle(sgmllib.SGMLParser):

    def __init__(self, verbose=0):
        sgmllib.SGMLParser.__init__(self, verbose)
        self.title = self.data = None

    def handle_data(self, data):
        if self.data is not None:
            self.data.append(data)

    def start_title(self, attrs):
        self.data = []

    def end_title(self):
        self.title = "".join(self.data)
        raise FoundTitle # abort parsing!

def extract(file):
    # extract title from an HTML/SGML stream
    p = ExtractTitle()
    try:
        while 1:
            # read small chunks
            s = file.read(512)
            if not s:
                break
            p.feed(s)
        p.close()
    except FoundTitle:
        return p.title
    return None

In [18]:
extract(open("N0001.sgml"))

'What is a Fenqing ?'

In [6]:
pd.__version__

'1.3.0'

In [10]:
from bs4 import BeautifulSoup

﻿<doc id='N0001'>
<head>
<title>What is a Fenqing ?</title>
<corpus url='http://code.google.com/p/evbcorpus/'>EVBCorpus</corpus>
<author email='hungnq@uit.edu.vn'>Quoc-Hung Ngo, Werner Winiwarter</author>
<citation>Quoc-Hung Ngo, Werner Winiwarter, 2012. "Building an English-Vietnamese Bilingual Corpus for Machine Translation", 
          In Proceedings of the International Conference on Asian Language Processing 2012 (IALP 2012), pp. 157-160.</citation>
<release>22/10/2013</release>
</head>
<text>
<spair id='1'>
<s id='en1'>What is a Fenqing ?</s>
<s id='vn1'>Fenqing là gì ?</s>
<a id='ev1'>1-3;2-2;4-1;</a>
</spair>
<spair id='2'>
<s id='en2'>Fenqing is a Chinese word which literally means " angry youth " .</s>
<s id='vn2'>Fenqing là một từ tiếng Hoa mà nghĩa đen là " thanh niên phẫn nộ " .</s>
<a id='ev2'>1-1;2-2;3-3;4-5,6;5-4;6-7;7-8,9;8-10;10-14,15;11-12,13;</a>
</spair>
<spair id='3'>
<s id='en3'>This word has many translations in English such as cynical youth , young nationalists

In [ ]:
sgml_files = Path('.').glob('*.sgml'

In [44]:
import re

def remove_punc(instring):
  outstring = re.sub(r'[^\w\s]', '', instring)
  return outstring  

sgml_files = Path('.').glob('*.sgml')
en = []
vn = []
for file in sgml_files:
  with open(file,'r') as f:
    doc = f.read()
    soup = BeautifulSoup(doc, 'html.parser')
    out = soup.find_all('spair')
    for i in range(len(out)):
      e = remove_punc(out[i].find('s', {'id':f'en{i+1}'}).getText())
      v = remove_punc(out[i].find('s', {'id':f'vn{i+1}'}).getText())
      if len(v.split()) <= 12:
        en.append(e)
        vn.append(v)
        # print(e, v)

In [45]:
len(en)
len(vn)

9884

In [51]:
vn[-70]

'Thực tiễn này đẩy chúng ta đến đâu '

In [28]:
out[50]

<spair id="51">
<s id="en51">We had a 3/4 week of it</s>
<s id="vn51">Chúng ta đã có 3/4 tuần</s>
<a id="ev51">1-1,2;2-3,4;4-5;5-6;</a>
</spair>

In [9]:
pd.read_xml("N0001.sgml")

,title,corpus,author,citation,release,spair
0,What is a Fenqing ?,EVBCorpus,"Quoc-Hung Ngo, Werner Winiwarter","Quoc-Hung Ngo, Werner Winiwarter, 2012. ""Build...",22/10/2013,NaN
1,None,None,None,None,None,NaN
